# AE2: Wypełnianie koła prostokątami
Adrianna Grudzień

Rozwiązanie wariantu problemu znanego w literaturze jako `cutting stock` problem.

Mamy dane koło o promieniu r oraz zbiór dostępnych prostokątów zadanych przez trzy liczby: wysokość, szerokość i wartość.

Celem jest ułożenie prostokątów w kole tak, aby zmaksymalizować sumę ich wartości, spełniając następujące warunki:

- boki wszystkich prostokątów były równoległe do osi układu,
- wnętrza prostokątów nie miały części wspólnej (intuicyjnie: prostokąty nie nachodzą na siebie, ale mogą się stykać bokami),
- każdy prostokąt można wstawić dowolnie wiele razy.


In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd

W nazwie pliku jest podany `promień koła`, plik jest w formacie csv: pierwsza kolumna to `szerokość` prostokąta, druga to `wysokość`, trzecia to `wartość` prostokąta.

In [5]:
r1000 = pd.read_csv('data/r1000.csv', header=None)
r1000

,0,1,2
0,200,120,200
1,200,160,300
2,250,160,500
3,100,120,40


# Opis koncepcji rozwiązania
Koło przechowuję w postaci macierzy (x,y), gdzie drewno jest oznaczone wartością 1, a brak drewna - 0. Po wycięciu prostokąta, obszar wycięty zmienia się z 1 na 0.

1. Inicjalizacja populacji.\
Najpierw obliczam pole koła i obliczam ile maksymalnie prostokątów się w nim zmieści (na podstawie pola powierzchni) - n. Następnie wybieram losowo 1 punkt z koła (wierzchołek jednego prostokąta). Szukam takiego ułożenie prostokąta, żeby mieścił się na obszarze drewnianym. Możliwych ułożeń jest 8. Sprawdzanie zaczynam od największego możliwego prostokąta - jeśli się nie mieści, sprawdzam coraz to mniejsze prostokąty.

2. Warunek stopu.\
Określony poziom znalezionego rozwiązania wynikający z kryteriów oceny zadania. To znaczy wartość sumaryczna prostokątów przynajmniej 30 000 w kole o średnicy 800 oraz 30 000 w kole o średnicy 1 200, minimum 17 500 w kole o średnicy 1 000 oraz minimum 25 000 w kole o średnicy 1 100.

3. Krzyżowanie.\
Losuję 2 rozwiązania z populacji. Nowego osobnika tworzę z prostokątów z górnej połowy pierwszego z nich oraz dolnej połowy drugiego z nich (tzn. biorę prostokąty, których wierzchołek się tam znajduje).

4. Mutacja.\
Jeżeli na kole jest jeszcze miejsce, to wstawiam tam prostokąt. Tzn. zaczynam od największego możliwego i jeśli po wylosowaniu okazuje się, że jednak się nie mieści na żaden z 8 sposobów, to losuję kolejny punkt dla tego prostokąta (i tak k razy). Jeśli po k takich losowań nie udało się, to testuję mniejsze prostokąty.

5. Ewaluacja.\
Liczę nagrodę, czyli zarobione z danego wycięcia pieniądze.

6. Selekcja\
W populacji zostawiam te wycięcia, które jednocześnie są najbardziej opłacalne pieniężnie i mają najwięcej wolnego miejsca w kole (po wycięciu), czyli te, w których zostało więcej niewykorzystanego materiału.

In [9]:
r1100 = pd.read_csv('data/r1100.csv', header=None)
r1200 = pd.read_csv('data/r1200.csv', header=None)
r800 = pd.read_csv('data/r800.csv', header=None)
r850 = pd.read_csv('data/r850.csv', header=None)